## Descripció

Aprèn a realitzar web scraping.

## Nivell 1

**- Exercici 1:**
Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import lxml
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import csv
from datetime import datetime
import re

#Exercici 2
from docx import Document
from docx.shared import Cm

#Exercici 3
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner
import json

# Reactor restart
from crochet import setup, wait_for
setup()

# text cleaning
import re


# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform

platform.python_version()

'3.9.12'

In [2]:
url_page = "http://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx"

page = requests.get(url_page)

In [3]:
page.status_code #si el codi és diferent a 200, la web no s'ha descarregat bé!

200

In [4]:
soup = BeautifulSoup(page.content, "html.parser") #genera arbre d'objectes que contenen la info

Practico una mica amb les funcionalitats de BeautifulSoup abans d'extreure valors de la taula:

In [5]:
print(soup.prettify()) #petrify() millora el format de la impressió

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Fri, 06 May 2022 09:13:08 GMT" data-idioma="esp">
  <meta content="IE=11" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Copyright © BME 2022" id="ctl00_copyright" name="copyright"/>
  <title>
   Bolsa de Madrid - Resumen de Índices
  </title>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" tit

In [6]:
list(soup.children) #elements at the top level

['\n',
 'html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"',
 '\n',
 <html xmlns="http://www.w3.org/1999/xhtml">
 <head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Fri, 06 May 2022 09:13:08 GMT" data-idioma="esp"><meta content="IE=11" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="Copyright © BME 2022" id="ctl00_copyright" name="copyright"/><title>
 	Bolsa de Madrid - Resumen de Índices
 </title><link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" title="Bolsa de Ma

In [7]:
print ('soup.title: ', soup.title)
print('soup.title.string: ', soup.title.string)
print('soup.p: ', soup.p)
print('soup.a: ', soup.a)

soup.title:  <title>
	Bolsa de Madrid - Resumen de Índices
</title>
soup.title.string:  
	Bolsa de Madrid - Resumen de Índices

soup.p:  None
soup.a:  <a href="/?id=ing" target="_self"> English </a>


In [8]:
soup.find_all('a') #tots els links de la pàgina:

[<a href="/?id=ing" target="_self"> English </a>,
 <a href="/esp/BMadrid/Contacto.aspx" target="_self"> Contacto </a>,
 <a href="/esp/Inversores/Agenda/HorarioMercado.aspx" target="_self"> Horario Mercado </a>,
 <a href="/esp/aspx/Inversores/Agenda/Calendario.aspx" target="_self"> Calendario bursátil </a>,
 <a href="/esp/RSS.aspx" target="_self"> RSS   <img align="absmiddle" alt="RSS" border="0" src="/images/IconoRSS.png"/> </a>,
 <a href="/?id=esp"><img alt="Bolsa de Madrid" border="0" src="/images/Base/LogoBMadrid.gif"/></a>,
 <a href="https://www.bolsasymercados.es/" target="_blank"><img alt="Bolsas y Mercados Españoles" border="0" height="45" src="/images/Base/LogoBMEBlanco.png?v=Six" width="118"/></a>,
 <a></a>,
 <a href="javascript:document.forms.formBusq.submitbusq();"><span class="BtnBuscarDcha" title="Buscar"></span></a>,
 <a href="/?id=esp" target="_self">Inicio</a>,
 <a href="#" target="_self">ÍNDICES</a>,
 <a href="#" target="_self">SOBRE NOSOTROS</a>,
 <a href="/esp/BMadri

In [9]:
for link in soup.find_all('a'): #els links de la pàgina que son http:
    if link.get ('href'):
        text_link=link.get('href')
        x=re.findall("\Ahttp", text_link )
        if x:
            print(link.get('href'))

https://www.bolsasymercados.es/
https://www.bmegrowth.es/
http://www.latibex.com
https://www.bolsasymercados.es/MTF_Equity/esp/SICAV/Precios.aspx?mercado=ValorLiquidativo
https://www.bolsasymercados.es/MTF_Equity/esp/SIL/Precios.aspx
https://www.bmerv.es/esp/aspx/Fondos/Portada/Portada.aspx
https://www.bolsasymercados.es/MTF_Equity/esp/ECR/Precios.aspx
https://www.latibex.com/esp/Indices/Informe-Mensual
http://www.latibex.com
http://www.bolsasymercados.es/esp/Estudios-Publicaciones/Estadisticas
http://www.bolsasymercados.es/esp/Estudios-Publicaciones/Informe-Mercado
http://www.bolsasymercados.es/esp/Estudios-Publicaciones/Informe-Mercado
https://www.bolsasymercados.es/esp/Estudios-Publicaciones
https://www.bolsasymercados.es/esp/Estudios-Publicaciones/Libros-y-Guias
http://biblioteca.bolsasymercados.es/opac/
https://www.bolsasymercados.es/esp/Estudios-Publicaciones/Libros-y-Guias
http://www.bolsasymercados.es/BME_4Companies/esp
http://www.entornopremercado.es
http://www.bolsasymercados

In [10]:
print(soup.get_text()) #extreu tot el text de la pàgina web





	Bolsa de Madrid - Resumen de Índices






 English 
 Contacto  Horario Mercado  Calendario bursátil  RSS    














Buscador deempresas












Inicio / ÍNDICES / Resumen de Índices >






SOBRE NOSOTROS

Bolsa de Madrid
Historia de la Bolsa
El Palacio de la Bolsa

Historia del Palacio
Visitas Guiadas
Visita Virtual
Actos y Conferencias
Aperturas de Honor

Formación

Cursos y Masters

Protector del Inversor

Información General
Consultas y Reclamaciones
Informes Anuales
Normativa

Miembros

Listado de miembros

Regulación
Cánones
Dónde estamos
Contacto

MERCADOS Y COTIZACIONES

Acciones

Precios de la sesión
Gráficos históricos

ETFs
Warrants
BME Growth
Latibex
BME IICs - Sicavs
BME IICs - Sil
BME IICs - Fondos
BME ECR - Ecr
Mi cartera

ÍNDICES

Resumen de Índices
Índices IBEX

Precios Sesión
Gráficos históricos
Precios históricos
Informes

Resumen Diario
Resumen Semanal
Informe Anual
Informe Mensual
Composición

Composición histórica
IBEX Growth Market®
Estrategia so

Procedirem a extreure la informació de la taula que es troba en aquesa pàgina web amb tres metodologies diferents:

## Pandas

La manera més senzilla de fer scraping per **extreure taules** és amb **pandas**, de tal manera que amb una sola línea de codi es pot extreure les taules que conté la pàgina:

In [11]:
dfs = pd.read_html(url_page)

In [12]:
len(dfs) 

2

Extreu dos taules, després d'una revisió veiem que la que ens interessa és la 2a:

In [13]:
dfs[1]

,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.Año 2022
0,IBEX 35®,"8.500,50","8.434,70",-077,"8.651,20","8.427,60",05/05/2022,17:38:00,-320
1,IBEX 35® con Dividendos,"26.594,40","26.388,70",-077,"27.065,90","26.366,40",05/05/2022,17:38:00,-182
2,IBEX MEDIUM CAP®,"13.059,10","12.967,10",-070,"13.223,50","12.961,60",05/05/2022,17:38:00,-614
3,IBEX SMALL CAP®,"8.513,90","8.470,50",-051,"8.668,40","8.470,50",05/05/2022,17:38:00,277
4,IBEX 35® Bancos,46710,45380,-285,47930,45360,05/05/2022,17:38:00,-199
...,...,...,...,...,...,...,...,...,...
80,Índice ITX Inverso X3,36970,34010,-801,35260,32100,05/05/2022,17:38:00,7531
81,Índice TEF Inverso X5,"2.514,10","2.316,60",-786,"2.474,50","2.303,30",05/05/2022,17:38:00,-7582
82,Índice SAN Inverso X5,"1.771,80","2.034,00",1480,"2.064,30","1.541,80",05/05/2022,17:38:00,-4815
83,Índice BBVA Inverso X5,"3.414,30","4.055,70",1879,"4.055,70","2.939,00",05/05/2022,17:38:00,-4243


Es pot treballar amb Pandas per obtenir i treballar amb la informació que conté.

## BeautifulSoup

D'altra banda es pot extreure la taula com objecte a través de la seva ID, però extreure'n la informació que conté és molt més farragós:

In [14]:
# Obtentim taula amb un ID específico
tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblÍndices'})
tabla.attrs

{'class': ['TblPort'],
 'cellspacing': '0',
 'cellpadding': '3',
 'align': 'Center',
 'id': 'ctl00_Contenido_tblÍndices',
 'width': '85%',
 'style': 'margin-bottom: 20px;'}

In [15]:
print(tabla.getText()) #es podria obtenir la info de la taula a partir del text també:



NombreAnteriorÚltimo% Dif.MáximoMínimoFechaHora% Dif.Año 2022

IBEX 35®8.500,508.434,70-0,778.651,208.427,6005/05/202217:38:00-3,20

IBEX 35® con Dividendos26.594,4026.388,70-0,7727.065,9026.366,4005/05/202217:38:00-1,82

IBEX MEDIUM CAP®13.059,1012.967,10-0,7013.223,5012.961,6005/05/202217:38:00-6,14

IBEX SMALL CAP®8.513,908.470,50-0,518.668,408.470,5005/05/202217:38:002,77

IBEX 35® Bancos467,10453,80-2,85479,30453,6005/05/202217:38:00-1,99

IBEX 35® Energía1.492,501.482,50-0,671.511,401.481,5005/05/202217:38:009,85

IBEX 35® Construcción1.593,501.578,20-0,961.622,001.576,7005/05/202217:38:00-1,13

IBEX Gender Equality8.893,308.859,40-0,389.023,008.849,2005/05/202217:38:001,71

IBEX Gender Equality Total Return9.970,109.932,00-0,3810.115,509.920,5005/05/202217:38:002,56

IBEX Gender Equality Net Return9.760,709.723,40-0,389.903,009.712,2005/05/202217:38:002,40

IBEX TOP Dividendo®2.682,002.678,30-0,142.715,002.677,2005/05/202217:38:007,19

IBEX 35® con Dividendos Netos22.169,8021.

In [16]:
columnes_noms=[]
for fila in tabla.find_all("th"):
    columnes_noms.append(fila.get_text())
    
columnes_noms
    

['Nombre',
 'Anterior',
 'Último',
 '% Dif.',
 'Máximo',
 'Mínimo',
 'Fecha',
 'Hora',
 '% Dif.Año 2022']

In [17]:
price=""
nroFila=0
for fila in tabla.find_all("tr"):
    if nroFila==0:
        for celda in fila.find_all('tr'):
            print(celda.text)
        
    if nroFila==1:
        nroCelda=0
        for celda in fila.find_all('td'):
            if nroCelda==0:
                name=celda.text
                print("Indice:", name)
            if nroCelda==2:
                price=celda.text
                print("Valor último:", price)
            nroCelda=nroCelda+1
    nroFila=nroFila+1

Indice: IBEX 35®
Valor último: 8.434,70


In [18]:
ibex_df1=pd.DataFrame(columns=columnes_noms) #generem dataframe a partir del text extret de la taula:
price=""
nroFila=0
for fila in tabla.find_all("tr"):
    if nroFila>0:#determina quina fila volem
        linea_valores=fila.find_all('td')
        nueva_fila = {'Nombre':linea_valores[0].text,
              'Anterior': linea_valores[1].text,
              'Último' : linea_valores[2].text, 
              '% Dif.' : linea_valores[3].text, 
              'Máximo' : linea_valores[4].text, 
              'Mínimo' : linea_valores[5].text,
              'Fecha' : linea_valores[6].text,
              'Hora' : linea_valores[7].text,
              '% Dif.\nAño 2022' : linea_valores[8].text,
        }           
            
        nueva_fila_df=pd.DataFrame([nueva_fila])
        ibex_df1=pd.concat([ibex_df1,nueva_fila_df],axis=0,ignore_index=True)
    nroFila+=1

In [19]:
ibex_df1

,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.Año 2022,% Dif.\nAño 2022
0,IBEX 35®,"8.500,50","8.434,70","-0,77","8.651,20","8.427,60",05/05/2022,17:38:00,NaN,"-3,20"
1,IBEX 35® con Dividendos,"26.594,40","26.388,70","-0,77","27.065,90","26.366,40",05/05/2022,17:38:00,NaN,"-1,82"
2,IBEX MEDIUM CAP®,"13.059,10","12.967,10","-0,70","13.223,50","12.961,60",05/05/2022,17:38:00,NaN,"-6,14"
3,IBEX SMALL CAP®,"8.513,90","8.470,50","-0,51","8.668,40","8.470,50",05/05/2022,17:38:00,NaN,"2,77"
4,IBEX 35® Bancos,"467,10","453,80","-2,85","479,30","453,60",05/05/2022,17:38:00,NaN,"-1,99"
...,...,...,...,...,...,...,...,...,...,...
80,Índice ITX Inverso X3,"369,70","340,10","-8,01","352,60","321,00",05/05/2022,17:38:00,NaN,"75,31"
81,Índice TEF Inverso X5,"2.514,10","2.316,60","-7,86","2.474,50","2.303,30",05/05/2022,17:38:00,NaN,"-75,82"
82,Índice SAN Inverso X5,"1.771,80","2.034,00","14,80","2.064,30","1.541,80",05/05/2022,17:38:00,NaN,"-48,15"
83,Índice BBVA Inverso X5,"3.414,30","4.055,70","18,79","4.055,70","2.939,00",05/05/2022,17:38:00,NaN,"-42,43"


## Selenium

Finalment fem servir Selenium per extreure la info de la taula:

In [20]:
driver = webdriver.Chrome('/Users/march/Downloads/chromedriver.exe')
driver.get(url_page)

C:\Users\march\AppData\Local\Temp\ipykernel_20344\3170386734.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/march/Downloads/chromedriver.exe')


In [21]:
cabecera=driver.find_elements_by_xpath('//th[@scope="col"]') # Extraiem el nom de cada columna
columnes_noms=[]
for i in range(len(cabecera)):
    columnes_noms.append(cabecera[i].text)
columnes_noms

C:\Users\march\AppData\Local\Temp\ipykernel_20344\1074450414.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cabecera=driver.find_elements_by_xpath('//th[@scope="col"]') # Extraiem el nom de cada columna


['Nombre',
 'Anterior',
 'Último',
 '% Dif.',
 'Máximo',
 'Mínimo',
 'Fecha',
 'Hora',
 '% Dif.\nAño 2022']

In [22]:
index_valors = driver.find_elements_by_xpath('//td[@class="DifFlBj"]') #exkraiem els valors
index_a_seguir=[]

for i in range(len(index_valors)):
    index_a_seguir.append(index_valors[i].text)

index_valors2 = driver.find_elements_by_xpath('//td[@class="DifFlSb"]') #exkraiem els valors
for i in range(len(index_valors2)):
    index_a_seguir.append(index_valors2[i].text)
    
print('index_a_seguir té:', len(index_a_seguir), 'entrades.')

C:\Users\march\AppData\Local\Temp\ipykernel_20344\1174506622.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  index_valors = driver.find_elements_by_xpath('//td[@class="DifFlBj"]') #exkraiem els valors
C:\Users\march\AppData\Local\Temp\ipykernel_20344\1174506622.py:7: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  index_valors2 = driver.find_elements_by_xpath('//td[@class="DifFlSb"]') #exkraiem els valors


index_a_seguir té: 84 entrades.


In [23]:
taula = driver.find_elements_by_id("ctl00_Contenido_tblÍndices")

C:\Users\march\AppData\Local\Temp\ipykernel_20344\2897043640.py:1: DeprecationWarning: find_elements_by_id is deprecated. Please use find_elements(by=By.ID, value=id_) instead
  taula = driver.find_elements_by_id("ctl00_Contenido_tblÍndices")


In [24]:
texto_tabla=(taula[0].text)
lista_texto_tabla=texto_tabla.split('\n')

In [25]:
columnes_noms

['Nombre',
 'Anterior',
 'Último',
 '% Dif.',
 'Máximo',
 'Mínimo',
 'Fecha',
 'Hora',
 '% Dif.\nAño 2022']

In [26]:
ibex_df=pd.DataFrame(columns=columnes_noms) #generem dataframe a partir del text extret de la taula:
texto_tabla=(taula[0].text)
lista_texto_tabla=texto_tabla.split('\n')
for i in index_a_seguir:
    res = [idx for idx in lista_texto_tabla if idx.startswith(i)] 
    datos_linea=res[0].split()
    nueva_fila = {'Nombre':i,
              'Anterior': datos_linea [-8] ,
              'Último' : datos_linea [-7], 
              '% Dif.' : datos_linea [-6], 
              'Máximo' : datos_linea [-5], 
              'Mínimo' : datos_linea [-4],
              'Fecha' : datos_linea [-3],
              'Hora' : datos_linea [-2],
              '% Dif.\nAño 2022' : datos_linea [-1],
    }

    ibex_df=ibex_df.append (nueva_fila , ignore_index=True)

C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.appe

C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.appe

C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.append (nueva_fila , ignore_index=True)
C:\Users\march\AppData\Local\Temp\ipykernel_20344\3699009988.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ibex_df=ibex_df.appe

In [27]:
ibex_df
     

,Nombre,Anterior,Último,% Dif.,Máximo,Mínimo,Fecha,Hora,% Dif.\nAño 2022
0,IBEX 35®,"8.500,50","8.434,70","-0,77","8.651,20","8.427,60",05/05/2022,17:38:00,"-3,20"
1,IBEX 35® con Dividendos,"26.594,40","26.388,70","-0,77","27.065,90","26.366,40",05/05/2022,17:38:00,"-1,82"
2,IBEX MEDIUM CAP®,"13.059,10","12.967,10","-0,70","13.223,50","12.961,60",05/05/2022,17:38:00,"-6,14"
3,IBEX SMALL CAP®,"8.513,90","8.470,50","-0,51","8.668,40","8.470,50",05/05/2022,17:38:00,"2,77"
4,IBEX 35® Bancos,"467,10","453,80","-2,85","479,30","453,60",05/05/2022,17:38:00,"-1,99"
...,...,...,...,...,...,...,...,...,...
79,Índice ITX Apalancado X5,"20,40","23,10","13,24","24,90","22,00",05/05/2022,17:38:00,"-84,95"
80,Índice SAN Inverso X3,"32,50","35,40","8,92","35,70","30,00",05/05/2022,17:38:00,"-18,62"
81,Índice BBVA Inverso X3,"27,00","30,00","11,11","30,00","24,70",05/05/2022,17:38:00,"-13,04"
82,Índice SAN Inverso X5,"1.771,80","2.034,00","14,80","2.064,30","1.541,80",05/05/2022,17:38:00,"-48,15"


In [28]:
driver.close()

webs recomanades al sprint 12:
* BEAUTIFUL SOUP 
* https://realpython.com/beautiful-soup-web-scraper-python/
* https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482  (Extra!)
SELENIUM
* https://realpython.com/modern-web-automation-with-python-and-selenium/
* https://www.selenium.dev/

ADVANCED WEB SCRAPPING
* https://towardsdatascience.com/everything-you-need-to-know-about-web-scraping-6541b241f27e
* https://scrapy.org/
* https://www.codementor.io/@scrapingdog/10-tips-to-avoid-getting-blocked-while-scraping-websites-16papipe62


https://towardsdatascience.com/web-scraping-with-beautiful-soup-selenium-or-scrapy-62c6f3545de7

https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a




## Nivell 2

**- Exercici 2:**
Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.


### Extraiem la informació amb BeautifulSoup i Selenium

In [211]:
url2='https://www.kaggle.com/datasets/muratkokludataset/acoustic-extinguisher-fire-dataset'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url2)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\march\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [212]:
web_title=driver.title

In [213]:
xpath='/html/body/main/div[1]/div/div[4]/div[3]/div[5]/div[1]/div[1]/div[2]'#Localització info inicial

intro_dataset=driver.find_elements(By.XPATH,value=xpath)[0].text
print(intro_dataset)

DATASET: https://www.muratkoklu.com/datasets/
CV:https://www.muratkoklu.com/en/publications/
Yavuz Selim TASPINAR, Murat KOKLU and Mustafa ALTIN
Citation Request :
1: KOKLU M., TASPINAR Y.S., (2021). Determining the Extinguishing Status of Fuel Flames With Sound Wave by Machine Learning Methods. IEEE Access, 9, pp.86207-86216, Doi: 10.1109/ACCESS.2021.3088612
Link: https://ieeexplore.ieee.org/document/9452168 (Open Access)
https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9452168
2: TASPINAR Y.S., KOKLU M., ALTIN M., (2021). Classification of Flame Extinction Based on Acoustic Oscillations using Artificial Intelligence Methods. Case Studies in Thermal Engineering, 28, 101561, Doi: 10.1016/j.csite.2021.101561
Link: https://www.sciencedirect.com/science/article/pii/S2214157X21007243 (Open Access) https://www.sciencedirect.com/sdfe/reader/pii/S2214157X21007243/pdf
3: TASPINAR Y.S., KOKLU M., ALTIN M., (2022). Acoustic-Driven Airflow Flame Extinguishing System Design and Analysis of

In [214]:
#Extracció noms dels arxius:
dataset = driver.find_elements(By.XPATH,value="//p[contains( text( ), 'Dataset')]")
file_dataset=[]
for i in range(len(dataset)) :
    if '.' in dataset[i].text:
        file_dataset.append(dataset[i].text)
file_dataset=set(file_dataset) #eliminem repeticions fent set
file_dataset

{'Acoustic_Extinguisher_Fire_Dataset.arff',
 'Acoustic_Extinguisher_Fire_Dataset.xlsx',
 'Acoustic_Extinguisher_Fire_Dataset_Citation_Request.txt'}

In [215]:
def text_in_list_obj(list_obj):
    text_list=[]
    for i in list_obj:
        if i.text!='':
            text_list.append (i.text)
    return text_list
def cut_string(string):
    words_string=string.split('\n')
    return words_string
    

In [208]:
#Extracció info del sumari
sumari_search=driver.find_elements(By.XPATH,value='/html/body/main/div[1]/div/div[4]/div[3]/div[5]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div')
summari_list_cut=[]
summari_list=text_in_list_obj(sumari_search)
for i in range (len(summari_list)):
    summari_list_cut.append(cut_string(summari_list[i]))
files_number= summari_list_cut[0][2]
files_number

'3 files'

In [124]:
#Extraiem el tamany de la descàrrega total del arxiu zip
download_search=driver.find_elements(By.XPATH, value='//*[@id="site-content"]/div[3]/div[2]/div/div[1]/div/a/button/span')                                    
size_files_to_download= download_search[0].text.split('(')[1].split(')')[0]

size_files_to_download

'636 kB'

In [126]:
#Extracció info del data explorer
file_occupancy=driver.find_elements(By.XPATH,value='/html/body/main/div[1]/div/div[4]/div[3]/div[5]/div[2]/div[2]/div/div[2]/div[1]/div/div/p')[0].text
file_occupancy

'Version 1 (1.05 MB)'

In [127]:
data_explorer=driver.find_elements(By.XPATH,value='//*[@id="site-content"]/div[3]/div[5]/div[2]/div[2]/div/div[2]/div[1]/div/ul/ul/ul')[0]
data_explorer_list=data_explorer.find_elements(By.TAG_NAME,'p')
name_files_list=[]
for i in range(len(data_explorer_list)):
    if data_explorer_list[i].text!='':
        name_files_list.append(data_explorer_list[i].text)
name_files_list

['Acoustic_Extinguisher_Fire_Dataset.arff',
 'Acoustic_Extinguisher_Fire_Dataset.xlsx',
 'Acoustic_Extinguisher_Fire_Dataset_Citation_Request.txt']

In [128]:
#Genera llista de llistes (una llista per cada arxiu excel que conté les fulles) 
files_tree=[]
for i in range(len(name_files_list)) :
    tree_excell=[]   
    if '.xlsx' in name_files_list[i]:
        z=i+1
        tree_excell.append(name_files_list[i])
        while (z >= 0) & (z <= len(name_files_list)):
            if '.' not in name_files_list[z]:
                tree_excell.append(name_files_list[z])
            else:    
                z=-2
            z+=1
        files_tree.append(tree_excell)
        
   

In [80]:
files_tree

[['Acoustic_Extinguisher_Fire_Dataset.xlsx',
  'A_E_Fire_Dataset',
  'Citation_Request',
  'Explanation']]

In [129]:
#scrap del web de descàrrega del dataset
download_link=driver.find_elements(By.XPATH,value='//*[@id="site-content"]/div[3]/div[4]/div[1]/div/div/div[2]/div/a')

if download_link!=[]:
    elems = download_link[0].find_elements(By.XPATH,value="//a[@href]")
    for elem in elems:
        if ('/download' in elem.get_attribute("href")) and ('/datasets/' in elem.get_attribute("href")):
            download_adress=(elem.get_attribute("href"))
download_adress




'https://www.kaggle.com/datasets/muratkokludataset/acoustic-extinguisher-fire-dataset/download'

In [131]:
#extraccio hashtacks
hashtack_list=[]
hashtack_pre = driver.find_elements(By.XPATH,value="//span[@class='sc-kLojOw sc-iklJeh eGLYxv hbusih']")
for i in range(len(hashtack_pre)):
    if hashtack_pre[i].text not in ['','Downloads']:
        hashtack_list.append (hashtack_pre[i].text)  
hashtack_list

['Computer Science',
 'Classification',
 'Categorical Data',
 'Artificial Intelligence',
 'Binary Classification']

In [240]:
#recerca taula info excel:
taula_excel=driver.find_elements(By.XPATH,value='/html/body/main/div[1]/div/div[4]/div[3]/div[5]/div[2]/div[2]/div/div[1]/div/div[3]/table/tbody')

taula_excel_columns=taula_excel[0].find_elements(By.TAG_NAME,'th') #recerca dels noms de les columnes
columnes_taula_excel=[]
for i in range(len(taula_excel_columns)):
    columnes_taula_excel.append(taula_excel_columns[i].text)
columnes_taula_excel

['Table', 'Total Rows', 'Total Columns']

In [259]:
taula_excel_contingut= taula_excel[0].find_elements(By.TAG_NAME,'tr')
table_content = []

for i in range(len(taula_excel_contingut)): #contingut de la taula
    if i>0:
        table_content.append(taula_excel_contingut[i].text.split(' '))    
table_content

[['A_E_Fire_Dataset', '17442', '7'],
 ['Citation_Request', '0', '1'],
 ['Explanation', '22', '4']]

In [262]:
taula_excel_df=pd.DataFrame(table_content, columns=columnes_taula_excel, index=None)
taula_excel_df

,Table,Total Rows,Total Columns
0,A_E_Fire_Dataset,17442,7
1,Citation_Request,0,1
2,Explanation,22,4


In [107]:
# AMB SOUP EXTRAUREM la short description
page2 = requests.get(url2)
page2.status_code #si el codi és diferent a 200, la web no s'ha descarregat bé!


200

In [108]:
soup2 = BeautifulSoup(page2.content, "html.parser") #genera arbre d'objectes que contenen la info

In [192]:
#Extracció del short description del data set
web_text=(soup2.prettify()) #petrify() millora el format de la impressió
short_description=web_text.split('SHORT DESCRIPTION:')[1].split('\\"\\t\\t\\t\\t\\n\\')[0]
NS_Description_list=new_short_description=short_description.split ('\\')
NS_Description_list= [x.lstrip('n') for x in NS_Description_list] 
NS_Description_list


[' The dataset was obtained as a result of the extinguishing tests of four different fuel flames with a sound wave extinguishing system. The sound wave fire-extinguishing system consists of 4 subwoofers with a total power of 4,000 Watt placed in the collimator cabinet. There are two amplifiers that enable the sound come to these subwoofers as boosted. Power supply that powers the system and filter circuit ensuring that the sound frequencies are properly transmitted to the system is located within the control unit. While computer is used as frequency source, anemometer was used to measure the airflow resulted from sound waves during the extinguishing phase of the flame, and a decibel meter to measure the sound intensity. An infrared thermometer was used to measure the temperature of the flame and the fuel can, and a camera is installed to detect the extinction time of the flame. A total of 17,442 tests were conducted with this experimental setup. The experiments are planned as follows:'

### Generem l'arxiu Word

In [322]:
document=Document()
document.add_heading(web_title, level=1)#afegim Títol

In [323]:
p0 = document.add_paragraph('\n\n\nThis web page is classified with the next hashtags: '+   ' / '.join(hashtack_list))

In [324]:
p = document.add_paragraph('\n\nLink to download the zip document which contains all data files: '+ download_adress)
p.add_run('Zip file size: '+size_files_to_download)
p.add_run('\n\nNumber of documents: '+files_number)
for i in range(len(name_files_list)):
    document.add_paragraph(name_files_list[i],style='List Number')


In [325]:
str_match = [s for s in name_files_list if s.__contains__(".xlsx")]
if str_match  !=[]:  
    p1b = document.add_paragraph ('\nThe file '+ str_match[0] + ' contains:\n')
    table = document.add_table(rows=taula_excel_df.shape[0]+1, cols=len(taula_excel_df.columns))

    for i in range(taula_excel_df.shape[0]):
        table.rows[0].cells[i].text = taula_excel_df.columns[i]
        for j in range(len(taula_excel_df.columns)):
            table.rows[i+1].cells[j].text = taula_excel_df.iloc[i,j]

In [326]:
p2=document.add_paragraph('\n\nShort description: \n').bold = True

In [327]:
p3=document.add_paragraph('\n '.join(NS_Description_list)) 

In [328]:
document.save('info_dataset.docx')

In [329]:
driver.close()

Generar documentació word:
* https://www.analyticslane.com/2020/09/07/creacion-de-documentos-word-con-python/ (Python-docx) 
* https://pharos.sh/lectura-y-escritura-de-archivos-de-ms-word-en-python-a-traves-del-modulo-python-docx/ (Python-docx)
Generar pdf i words:
* https://programmerclick.com/article/5297528417/

## Nivell 3

 **- Exercici 3:** Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 

## Scrapping amb la llibreria Scrapy

In [28]:


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('resultados.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item



class redditCryptoSpider(scrapy.Spider):
    name = "trendy_forums"
    start_urls = [
        'https://www.reddit.com/r/CryptoCurrency/hot/',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'reddit_criptos_top_forums.json'          # Used for pipeline 2
    }
    
    
    def parse(self, response):
        #Extracting the content using xpath 
        titles = response.xpath('//h3[@class="_eYtD2XCVieq6emjKBH3m"]/text()').getall()
        links = response.xpath('//a[@class="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE"]//@href').getall()
        links=['https://www.reddit.com'+x for x in links]
        number_comments = response.xpath('//span[@class="FHCV02u6Cp2zYL0fhQPsO"]//text()').getall()
        initiation = response.xpath('//a[@class="_3jOxDPIQ0KaOWpzvSQo-1s"]//text()').getall()
        for i in range(len(titles)):
            yield {
                    'Title': titles[i],
                    'Link': links[i],
                    'number of comment': number_comments[i],
                    'Initiation': initiation[i],
                }
   

In [31]:
process = CrawlerRunner()

process.crawl(redditCryptoSpider)

2022-05-09 17:33:09 [scrapy.crawler] INFO: Overridden settings:
{}
2022-05-09 17:33:09 [scrapy.extensions.telnet] INFO: Telnet Password: 8f84312170292257
2022-05-09 17:33:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-09 17:33:09 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'sc

## Lectura dels resultats en un DataFrame de Pandas

In [33]:
ruta_archivo_json = 'C:/Users/march/000 Treball Ju`yter 000/Sprints_IT Academy_Data Science/Sprint_12/reddit_criptos_top_forums.json'
reddit_criptos_top_forums_df = pd.read_json(ruta_archivo_json, orient='records')
reddit_criptos_top_forums_df

,Title,Link,number of comment,Initiation
0,r/CC Cointest - New Coin Inquiries round - May...,https://www.reddit.com/r/CryptoCurrency/commen...,25 comments,1 day ago
1,"Daily Discussion - May 9, 2022 (GMT+0)",https://www.reddit.com/r/CryptoCurrency/commen...,4.1k comments,16 hours ago
2,Crypto market crash is probably the best oppor...,https://www.reddit.com/r/CryptoCurrency/commen...,513 comments,4 hours ago
3,"It's not just Crypto, the whole stock market i...",https://www.reddit.com/r/CryptoCurrency/commen...,141 comments,5 hours ago
4,Some advice from a previous bear market partic...,https://www.reddit.com/r/CryptoCurrency/commen...,342 comments,14 hours ago
5,"What we are experiencing now, is the start of ...",https://www.reddit.com/r/CryptoCurrency/commen...,136 comments,3 hours ago


Font: 
* https://www.analyticsvidhya.com/blog/2017/07/web-scraping-in-python-using-scrapy/
* https://towardsdatascience.com/run-scrapy-code-from-jupyter-notebook-without-issues-69b7cb79530c
* https://josefgonzalez.me/es/post/scrapy-en-jupyter/